Dans un premier temps, on récupère les données et on les met dans une forme acceptable pour travailler sur elles

A faire : Avoir la bonne BDD car pour l'instant je n'ai que 5 observations sur id_text

In [22]:
import pandas as pd


def extraire_id_texte(chemin_fichier):
    #Hyp : Le chemin d'entrée doit mené à un fichier csv d'articles avec comme colonnes : id et text
    #Prend un chemin d'accès en entrée et retourne le dataframe correspondant
    
    df = pd.read_csv(chemin_fichier)
    
    return df



file_path = 'id_text.csv'
data = extraire_id_texte(file_path)


FileNotFoundError: [Errno 2] No such file or directory: 'id_text.csv'

In [27]:
def description_data(data, nbre_observ = 5):
    #Hyp : data est censé être un dataframe d'articles de presse
    #Cette fonction affiche quelques informations desriptives sur data : nombre d'observations, de variables et elle affiche en détails les 10 premières observations

    print(f"Nombre de variables : {data.shape[1]}")
    print(f"Nombre d'observations : {data.shape[0]}")
    print("\nPremières observations :")
    print(data.head(nbre_observ).to_string(index=False))  #Affiche tout le texte sans couper


Dans un premier temps, on récupère les articles de journaux. Pour cela, on s'aide des fonctions du fichier gettingNewspapers qui nous donne des fichiers en tar.gz contenant les articles par année de publication.

Dans un premier temps, on se familiarise avec ces fichiers, donc pour l'instant, on regarde uniquement l'année 1919 et on se limite aux premiers fichiers json pour essayer de bien comprendre la structure des fichiers tar.gz


Une fois que l'on aura bien compris tout cela et qu'on aura vérifié nos fonctions d'analyse, on pourra étendre l'analyse à une part plus importante du corpus de texte


In [6]:
import os

file = os.listdir(f"ArticlesTarGz/faro_{1919}/mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_{1919}")

fichier = json.load(open(f"ArticlesTarGz/faro_{1919}/mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_{1919}/" + file[0]))

print(fichier["lccn"])
print(fichier["full articles"][1])
print(fichier["full articles"][2])
print(fichier["full articles"][3])
print(fichier["full articles"][4])
print(fichier["full articles"][5])


{'title': 'The daily morning oasis.', 'geonames_ids': ['5306611'], 'dbpedia_ids': ['Nogales%2C_Arizona'], 'issn': '2692-7187', 'lccn': 'sn83025458', 'start_year': '1917', 'end_year': '1920', 'languages': ['English'], 'succeeds': [], 'successors': [], 'editions': []}
{'object_ids': [9, 2], 'headline': 'TEXAS OIL FIELDS LOOK\n\n\nVERY PROMISING', 'article': "FORT WORTH, Texas, Oct g.\nNorth Texas oil operators are show\ning considerable interest in what pro-\nmisses to be a new production area in\nthe Burkburnett oil field. The Tex\nhoma Oil Company has drilled i'm a\nshallow well about one and one-half\nmile south of the oil Burkburnett\ntownsite pool.\n\n\nThe well is approximately 1000\nfeet deep, about 500 feet shallower\nthan the townsite and Burk- Wag-\ngoner drilling and flowing sixty bar\nrels hourly.\n\n\nThis is considerer very good for a\nshallow pool. In case it develops by\nfuture drillings that new production\narea actually has been discovered.\nBurkburnett will again be th

In [ ]:
file = os.listdir(f"ArticlesTarGz/faro_{1919}/mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_{1919}")

print(file)

In [11]:
import os

file = os.listdir(f"ArticlesTarGz/faro_{1919}/mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_{1919}")

i = 0
while i !=1000:
    fichier = json.load(open(f"ArticlesTarGz/faro_{1919}/mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_{1919}/" + file[i]))
    print("\nFichier n°", i, " : ")
    print(fichier["lccn"])
    i = i+1
    






Fichier n° 0  : 
{'title': 'The daily morning oasis.', 'geonames_ids': ['5306611'], 'dbpedia_ids': ['Nogales%2C_Arizona'], 'issn': '2692-7187', 'lccn': 'sn83025458', 'start_year': '1917', 'end_year': '1920', 'languages': ['English'], 'succeeds': [], 'successors': [], 'editions': []}

Fichier n° 1  : 


KeyError: 'lccn'

In [2]:
import tarfile
import pandas as pd
import json

def charger_targz(chemin_archive, limite):
    #Hyp : chemin_archive mène à un fichier tar.gz (en particulier, les fichiers résultats des commandes de gettingNewspapers)
    #Hyp : limite est un nombre qui a pour but de limiter le nombre de fichier json que l'on regarde dans notre fichier tar.gz.

    #Renvoie un dataframe contenant les données extraites des fichiers json du fichier tar.gz

    donnees = []
    compteur = 0 

    with tarfile.open(chemin_archive, "r:gz") as archive:
        for fichier in archive.getnames():
            if fichier.endswith('.json'): 
                print(f"Lecture du fichier : {fichier}")
                extrait = archive.extractfile(fichier)
                if extrait:
                    try:
                        contenu_json = json.load(extrait) 
                        donnees.append(contenu_json)
                        compteur += 1
                        
                        if compteur >= limite:  # On met ici une limite potentielle car le fichier est très lourd (plus de 2 Go)
                            print(f"Limite de {limite} fichiers atteinte.")
                            break
                    except json.JSONDecodeError as e:
                        print(f"Erreur de lecture JSON dans le fichier {fichier} : {e}")
    
    return pd.DataFrame(donnees)

In [2]:
chemin_archive = "ArticlesTarGz/faro_1919.tar.gz"
data_brute_1919 = charger_targz(chemin_archive, limite=10)
print(data_brute_1919.head())
print(f"Nombre total d'observations chargées : {len(data_brute_1919)}")

Lecture du fichier : mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_1919/1919-04-24_p22_sn84026749_00280764577_1919042401_0444.json
Lecture du fichier : mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_1919/1919-04-20_p18_sn83045433_00280764024_1919042001_0068.json
Lecture du fichier : mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_1919/1919-10-04_p4_sn84020558_00295869823_1919100401_0336.json
Lecture du fichier : mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_1919/1919-06-21_p4_sn86063381_0028076727A_1919062101_1223.json
Lecture du fichier : mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_1919/1919-12-06_p1_sn85025007_00415664151_1919120601_0475.json
Lecture du fichier : mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_clean/faro_1919/1919-01-31_p2_sn95060791_00212476328_1919013101_0130.json
Lecture du fichier : mnt/122a7683-fa4b-45dd-9f13-b18cc4f4a187/ca_rule_based_fa_c

Quelques fonctions pour essayer de comprendre à quoi ressemble notre dataframe


In [3]:
pd.set_option('display.max_colwidth', None)  # Afficher le texte complet
print(data_brute_1919.head(1))

                                                                                                                                                                                                                                                                               lccn  \
0  {'title': 'The Washington times.', 'geonames_ids': [], 'dbpedia_ids': [], 'issn': '1941-0697', 'lccn': 'sn84026749', 'start_year': '1902', 'end_year': '1939', 'languages': ['English'], 'succeeds': ['sn84024441'], 'successors': ['sn85040694', 'sn83045433'], 'editions': []}   

                                                                                                                                                                                                                                                                                                              edition  \
0  {'lccn': 'sn84026749', 'edition': 'ed-01', 'date': '1919-04-24', 'pages': ['seq-12', 'seq-3', 'seq-8', 'seq-15', 'seq-2', 'se

In [ ]:
print()

In [14]:

print(data_brute_1919.columns) 

Index(['lccn', 'edition', 'page_number', 'scan', 'bboxes', 'full articles',
       'scan_url', 'scan_ocr'],
      dtype='object')


In [16]:
print(type(data_brute_1919["edition"][0]))
print(data_brute_1919["edition"][0].keys())


<class 'dict'>
dict_keys(['lccn', 'edition', 'date', 'pages'])


In [18]:
lccn_explose = pd.json_normalize(data_brute_1919['edition'].dropna())  # Extraire les champs JSON
print(lccn_explose.head())

         lccn edition        date  \
0  sn84026749   ed-01  1919-04-24   
1  sn83045433   ed-01  1919-04-20   
2  sn84020558   ed-01  1919-10-04   
3  sn86063381   ed-01  1919-06-21   
4         NaN     NaN  1919-12-06   

                                                                                                                                                                                                                                                                                                     pages  
0                                                                                                                  [seq-12, seq-3, seq-8, seq-15, seq-2, seq-7, seq-18, seq-23, seq-5, seq-6, seq-9, seq-10, seq-22, seq-16, seq-21, seq-24, seq-13, seq-14, seq-11, seq-4, seq-19, seq-1, seq-20, seq-17]  
1  [seq-10, seq-30, seq-37, seq-34, seq-19, seq-7, seq-26, seq-11, seq-38, seq-36, seq-20, seq-31, seq-21, seq-13, seq-35, seq-4, seq-28, seq-17, seq-2, seq-22, seq-1, seq-27,

In [15]:
lccn_explose = pd.json_normalize(data_brute_1919['lccn'].dropna())  # Extraire les champs JSON
print(lccn_explose.head())

                        title geonames_ids             dbpedia_ids       issn  \
0       The Washington times.           []                      []  1941-0697   
1      The Washington herald.           []                      []  1941-0662   
2         Arizona republican.    [5308655]    [Phoenix%2C_Arizona]  2157-135X   
3  The Bemidji daily pioneer.           []  [Bemidji%2C_Minnesota]  2163-4831   
4          The Holbrook news.    [5284580]   [Holbrook%2C_Arizona]  2375-1762   

         lccn start_year end_year  languages      succeeds  \
0  sn84026749       1902     1939  [English]  [sn84024441]   
1  sn83045433       1906     1939  [English]            []   
2  sn84020558       1890     1930  [English]            []   
3  sn86063381       1904     1971  [English]  [sn90059049]   
4  sn95060791       1909     1923  [English]            []   

                 successors editions  
0  [sn85040694, sn83045433]       []  
1  [sn85040694, sn84026749]       []  
2              [sn83045

In [16]:
import pandas as pd

# Étape 1 : Extraire les informations de 'lccn' et 'edition'
lccn_explose = pd.json_normalize(data_brute_1919['lccn'].dropna())  # Dérouler les champs JSON imbriqués
edition_explose = pd.json_normalize(data_brute_1919['edition'].dropna())  # Pareil pour 'edition'

# Étape 2 : Ajouter les colonnes d'intérêt au DataFrame d'origine
# Fusionner les données extraites (lccn_explose et edition_explose) avec l'index
data_brute_1919 = data_brute_1919.reset_index()
data_brute_1919 = data_brute_1919.join(lccn_explose, how='left', rsuffix='_lccn')
data_brute_1919 = data_brute_1919.join(edition_explose, how='left', rsuffix='_edition')

# Étape 3 : Sélectionner les colonnes d'intérêt et renommer
data_texte_1919 = df[['index', 'title', 'start_year', 'end_year', 'date', 'full articles']].copy()
data_texte_1919.columns = ['id', 'nom_journal', 'date_debut_publication', 'date_fin_publication', 'date_article', 'texte_article']



In [28]:
#On utilise notre fonction de description pour voir l'état de la BDD
description_data(data_texte_1919, 1)



Nombre de variables : 6
Nombre d'observations : 10

Premières observations :
 id           nom_journal date_debut_publication date_fin_publication date_article                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [85]:
print(data_texte_1919["texte_article"][0])

[{'object_ids': [38, 0], 'headline': 'RELATIVES NAY GET BODIES\n\n\nOF SOLDIERS FROM FRANCE', 'article': 'Bodies Of American soldiers killed\nIn action are being collected and in\nTERED in American cemeteries scat\ntered all along the battle lines IN\nFrance. former Attorney General\nGregory announces, upon his return\nfrom Paris.\n\n\nSome cemeteries contain only a\ndozen bodies In others the bodies\ntotal hundreds Every care Is being\ntaken to preserve the Identification\nOF the men buried and TO keep the\ngraves in good condition and preserve\nthe cemeteries sites from desecra-\nton.\n\n\nwhether the United States Govern\nment will adopt a policy OF returning\nAll the bodies to the United States\nwill be determined later, Gregory\nsaid. Meanwhile the temporary ex-\npedient OF laying out cemeteries has\nbeen adopted. Relatives in the Uni-\nted States who may wish to return\nbodies of soldiers TO this country will\nbe obliged to take up the matter with\nthe Government, which will act\

En attendant d'avoir la bonne base de données, j'en créé une à la main (le premier texte est un texte du Washington Post qui parle d'inflation)

In [17]:


def creer_bdd_fictive(textes):
    #Hyp : textes doit être une liste de chaîne de caractères

    #Renvoie un dataframe fictif sous le même format que id_text

    ids = [f"{i+1}" for i in range(len(textes))]
    df = pd.DataFrame({'id': ids, 'text': textes})
    
    return df


textes_fictif = [
    'President Joe Biden is commuting the sentences of nearly 1,500 people and pardoning 39 more convicted of nonviolent crimes, the White House said in a statement Thursday, describing it as “the largest single-day grant of clemency in modern history. Get the latest election news and results The individuals whose sentences will be commuted were placed on home confinement during the coronavirus pandemic and have “successfully reintegrated” with their families and communities, the White House said in a statement. The White House added that Biden is still reviewing clemency petitions and that more announcements are to come. Pressure has grown on the outgoing president to exercise his powers before he cedes the White House to President-elect Donald Trump, and opponents of capital punishment have called for Biden to empty federal death row. Trump staunchly supports capital punishment. Among those who received clemency Thursday are people who would have received a lighter sentence if they were charged under current laws and practice, Biden said in a statement. “America was built on the promise of possibility and second chances,” Biden said. The decision to grant mass clemency comes after backlash over Biden’s decision to pardon his son Hunter after he had previously said he would not. Part of the uproar came from criminal justice advocates, as well as fellow Democrats, who expressed frustration that Biden benefited his son alone, instead of broadly using his clemency power for less privileged people. Anita Dunn, a former senior adviser to the president who left the administration in August, said in a New York Times’ DealBook Summit posted Wednesday that she disagreed with the rollout of the pardon for Biden’s son. “I absolutely think that Hunter deserves a pardon here, but I disagree on the timing, the argument and sort of the rationale,” she said. Biden is also said to be weighing preemptive pardons for people who may be targeted by the incoming Trump administration, including Anthony S. Fauci, who helped coordinate the nation’s coronavirus response; retired Gen. Mark A. Milley, former chairman of the Joint Chiefs of Staff, who has called Trump a “fascist”; Sen.-elect Adam Schiff (D-California), who led the first impeachment effort against Trump; and former congresswoman Liz Cheney (R-Wyoming), an outspoken GOP critic of Trump. The 39 people pardoned Thursday were convicted of nonviolent crimes, including drug offenses. They “have turned their lives around,” the White House said. “These individuals are parents, veterans, health care professionals, teachers, advocates, and engaged members of their communities.” In the final days of his second term in 2017, President Barack Obama granted 330 commutations to nonviolent drug offenders — which at the time was the most granted in one day in U.S. history. Before Thursday’s announcement, Biden had commuted the sentences of over 140 people and granted 26 pardons in office — including most recently to his son Hunter. Biden has also granted pardons for military veterans convicted of having gay sex, once a crime under military law, and people convicted of marijuana possession at the federal level. Biden’s administration received nearly 12,000 petitions for clemency and more than 1,400 petitions for pardons through early December, according to statistics released by the Justice Department. Mark Berman, Ann E. Marimow, Yasmeen Abutaleb and Matt Viser contributed to this report..',
    'Texte vide.',
    'Inflation, price increase.',
]

bdd_fictive = creer_bdd_fictive(textes_fictif)
print(bdd_fictive)

  id                                               text
0  1  President Joe Biden is commuting the sentences...
1  2                                        Texte vide.
2  3                         Inflation, price increase.


Une des principales tâches est d'avoir un dictionnaire avec les expressions relatives à l'inflation et aux prix. On construit ici à la main un dictionnaire de 1-gram, 2-gram et 3-gram

In [54]:
def creer_dictionnaire_inflation():
    #Cette fonction renvoie un dictionnaire de n-gram relatifs à l'inflation et aux prix

    dictionnaire = {
        "1-gram": [
            "inflation", "deflation", "prices", "cost", "wages", "currency",
            "money", "devaluation", "economy", "market", "increase", "decrease"
        ],
        "2-gram": [
            "price level", "wage growth", "economic downturn", "monetary policy",
            "cost increase", "cost reduction", "market prices", "inflation rate",
            "interest rates", "price stability"
        ],
        "3-gram": [
            "consumer price index", "rise in prices", "fall in prices",
            "cost of living", "inflation expectations", "money supply growth",
            "central bank policy", "economic price adjustments"
        ]
    }
    return dictionnaire

dictionnaire_inflation = creer_dictionnaire_inflation()


A cette étape, nous avons notre dataframe, et notre dictionnaire. La prochaine étape et de récupérer dans chaque article l'ensemble des n-gram qu'il contient. Pour cela, il faut d'abord nettoyer les textes : enlever les majuscules, toutes parties qui n'est pas du texte. C'est ce que nous faisons avec les fonctions ci-dessous

In [48]:
import re

def nettoyer_texte(text):
    #Renvoie le texte text nettoyé, c'est-à-dire sans majuscule et avec uniquement les mots du texte (en particulier, on enlève les parties entre accolades qui semblent ne pas être du texte)
    text = text.lower()
    text = re.sub(r'\{.*?\}', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text) 
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [75]:
text_test_1 = "Miam le   price level        economic  price    adjustments     espace !!"
text_test_2 = "Texte, wouah inflation... { bla bla bizarre x1 : 258}     ."
text_test_3 = "normalent on ne change rien ici"
text_test_4 = " Edward II. and Leila Clemens, boy.Robert L. and Priscilla Craney, girl.Augustus Z. and Daisy Berry, girlJessie and Mary Anderson, boy.', 'byline': '', 'bbox_list': [{'x0': 2737, 'y0': 961, 'x1': 3267, 'y1': 1020}, {'x0': 2608, 'y0': 1038, 'x1': 3338, 'y1': 1433}], 'bbox': [2608, 961, 3338, 1433], 'full_article_id': 19, 'id': '19_1919-04-24_p22_sn84026749_00280764577_1919042401_0444.json'}, {'object_ids': [59, 19], 'headline': 'DOROTHY c. GILMORE.', 'article': 'With scores Of schoolmates andfriends present, funeral services arebeing held this afternoon for DorothyCaroline Gilmore, twelve years old.daughter Of Charles w. Gilmore,"

print(nettoyer_texte(text_test_1))
print(nettoyer_texte(text_test_2))
print(nettoyer_texte(text_test_3))
print(nettoyer_texte(text_test_4))

miam le price level economic price adjustments espace
texte wouah inflation
normalent on ne change rien ici
edward ii and leila clemens boy robert l and priscilla craney girl augustus z and daisy berry girljessie and mary anderson boy byline bbox list bbox full article id id p sn json object ids headline dorothy c gilmore article with scores of schoolmates andfriends present funeral services arebeing held this afternoon for dorothycaroline gilmore twelve years old daughter of charles w gilmore


In [79]:
def calculer_ratio_expression_inflation(text, inflation_dict):
    #Hyp : inflation_dict est un dictionnaire de n-gram lié aux termes d'inflation et de prix (en particulier, il doit avoir le même format que le dictionnaire en sorti de la fonction creer_dictionnaire_inflation())
    #Retourne un indice d'importance du sujet de l'inflation dans le texte text. On retourne plus exactement le ratio nombre d'expression liée à l'inflation / nombre de mots ainsi que le nombre d'expression liée à l'inflation
    
    text = nettoyer_texte(text)

    nbre_expression = 0
    nbre_total = len(text.split())

    coefficient = 4 #Ici, on donne plus d'importance aux n-gram plus longs

    for categorie_n_gram in ["3-gram", "2-gram", "1-gram"]:  
        coefficient = coefficient -1
        for expression in inflation_dict[categorie_n_gram]:
            expression2 = re.escape(expression)
            occurence_expression = re.findall(r'\b' + expression2 + r'\b', text)
            nbre_expression += len(occurence_expression) * coefficient

    # Dans le cas où le texte est vide (on évite de diviser par 0...)
    if nbre_total == 0:
        return 0.0, 0

    ratio = nbre_expression / nbre_total

    return ratio, nbre_expression

In [82]:
print(calculer_ratio_expression_inflation(text_test_1, dictionnaire_inflation))
print(calculer_ratio_expression_inflation(text_test_2, dictionnaire_inflation))
print(calculer_ratio_expression_inflation(text_test_4, dictionnaire_inflation))

(0.625, 5)
(0.3333333333333333, 1)
(0.0, 0)


On applique la fonction de calcul de ratio à l'ensemble des articles de la base de données et on associe les ratios à chaque texte en ajoutant deux colonnes sur note dataframe

In [84]:
def apply_inflation_ratio(text):
    ratio, nbre_expression = calculate_inflation_ratio(text, dictionnaire_inflation)
    return pd.Series([ratio, nbre_expression])

data_texte_1919_avec_ratio = data_texte_1919[['inflation_ratio', 'inflation_expression_count']] = data_texte_1919['texte_article'].apply(apply_inflation_ratio)


AttributeError: 'list' object has no attribute 'lower'

Brouillon : 

Accumulateur (compteur)
triplet (lccn titre, edition date, full articles (liste de dictionnaires))

renvoie le nouvel accumulateur 



Accumulateur : Compteur pour avancer dans la liste de texte, dictionnaire avec en clé les mois (pour classer par mois de chaque année les textes) et comme valeur une liste avec le nombre de texte parlant d'inflation et le nombre de texte total